In [6]:
import pandas as pd

In [7]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [8]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [9]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [10]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [11]:
dataset=dataset.drop('User ID',axis=1)

In [12]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [13]:
independent=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
dependent=dataset[['Purchased']]

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [15]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [17]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
param_grid={'solver':['newton-cg','lgbs','liblinear','saga'],
            'penalty':['l2']}

LGgrid=GridSearchCV(LogisticRegression(),param_grid,refit=True,verbose=2,n_jobs=1,scoring='f1_weighted')
LGgrid=LGgrid.fit(x_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END .......................penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END .......................penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END .......................penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END .......................penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END .......................penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END ............................penalty=l2, solver=lgbs; total time=   0.0s
[CV] END ............................penalty=l2, solver=lgbs; total time=   0.0s
[CV] END ............................penalty=l2, solver=lgbs; total time=   0.0s
[CV] END ............................penalty=l2, solver=lgbs; total time=   0.0s
[CV] END ............................penalty=l2, solver=lgbs; total time=   0.0s
[CV] END .......................penalty=l2, solver=liblinear; total time=   0.0s
[CV] END .......................penalty=l2, solve

D:\Anaconda3\envs\Kathir\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\envs\Kathir\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\envs\Kathir\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\envs\Kathir\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

In [25]:
re=LGgrid.cv_results_
LGgrid_pred=LGgrid.predict(x_test)

In [26]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,LGgrid_pred)

In [27]:
cm

array([[74,  5],
       [ 8, 33]])

In [29]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,LGgrid_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120



In [31]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_pred,average='weighted')
print('The f1 macro value from best parameters{}:'.format(LGgrid.best_params_),f1_macro)

The f1 macro value from best parameters{'penalty': 'l2', 'solver': 'newton-cg'}: 0.8906190214115365


In [32]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,LGgrid.predict_proba(x_test)[:,1])

np.float64(0.9481321395492436)

In [37]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005082,0.003226,0.008275,0.001582,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
1,0.001015,0.001552,0.000000,0.000000,l2,lgbs,"{'penalty': 'l2', 'solver': 'lgbs'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2,0.000463,0.000575,0.007420,0.000794,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
3,0.000289,0.000578,0.007700,0.000832,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
